In [1]:
! pip install requests


In [2]:
! pip install openai


In [3]:
! pip install python-dotenv

In [10]:
import requests
weather = requests.post("http://backend:8000/get_temperature",json={
  "location": "karaChi"
})
weather.json()

'{"location": "Karachi", "temperature": "10", "unit": "celsius"}'

In [16]:
from openai import  OpenAI
from dotenv import find_dotenv, load_dotenv
import os
_:bool = load_dotenv(find_dotenv())

client:OpenAI = OpenAI()

In [13]:
def get_temp_from_fastapi(location:str):
    url = "http://backend:8000/get_temperature"
    payload = {
        "location": location
    }
    weather = requests.post("http://backend:8000/get_temperature",json=payload)
    return weather.json()
get_temp_from_fastapi("Islamabad")

'{"location": "Islamabad", "temperature": "72", "unit": "fahrenheit"}'

In [19]:
import requests
from openai.types.chat.chat_completion import ChatCompletion
import json 

def get_weather_through_fastapi(location:str):
    url = "http://backend:8000/get_temperature"
    payload = {
        "location": location
    }
    weather = requests.post("http://backend:8000/get_temperature",json=payload)
    return weather.json()

def weather_bot(prompt:str)-> str:
    messages = [
            {
                "role":"user",
                "content":prompt
            }
        ]
    tools = [
            {
            "type": "function",
            "function": {
                "name": "get_weather_through_fastapi",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. lahore, karachi",
                        },
                    },
                    "required": ["location"],
                    },
                },
            }
        ]
    response_1: ChatCompletion = client.chat.completions.create(
        messages= messages,
        model = "gpt-3.5-turbo-1106",
        tools = tools,
        tool_choice ="auto" 
    )
    response_1_msg = response_1.choices[0].message
    response_1_tool_calls = response_1_msg.tool_calls
    print(f"Response 1 tool calls: {list(response_1_tool_calls)}")

    if response_1_tool_calls:
        available_functions = {
            "get_weather_through_fastapi" : get_weather_through_fastapi
        }
        messages.append(response_1_msg)

        for tool_call in response_1_tool_calls:
            function_name = str(tool_call.function.name)
            function_to_call = available_functions.get(function_name)
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                function_args.get("location")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id , 
                    "role":"tool",
                    "name":function_name,
                    "content": function_response
                }
            )
    print(f"messages to get Second response  : {messages}")
    second_response: ChatCompletion = client.chat.completions.create(
        messages = messages,
        model = "gpt-3.5-turbo-1106"
    )
    display((f"Second response  : {dict(second_response)}"))
    return second_response.choices[0].message.content



        
    




In [20]:
weather_bot("what's the weather like in Islamabad")

Response 1 tool calls: [ChatCompletionMessageToolCall(id='call_XNxyJxvECidp6LN5jByRUaLh', function=Function(arguments='{"location":"Islamabad"}', name='get_weather_through_fastapi'), type='function')]
messages to get Second response  : [{'role': 'user', 'content': "what's the weather like in Islamabad"}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XNxyJxvECidp6LN5jByRUaLh', function=Function(arguments='{"location":"Islamabad"}', name='get_weather_through_fastapi'), type='function')]), {'tool_call_id': 'call_XNxyJxvECidp6LN5jByRUaLh', 'role': 'tool', 'name': 'get_weather_through_fastapi', 'content': '{"location": "Islamabad", "temperature": "72", "unit": "fahrenheit"}'}]


"Second response  : {'id': 'chatcmpl-9ZEK7yZc665GFKjrIjozM3Ntj1gUK', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Currently, the weather in Islamabad is 72 degrees Fahrenheit.', role='assistant', function_call=None, tool_calls=None))], 'created': 1718183567, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_482d920018', 'usage': CompletionUsage(completion_tokens=12, prompt_tokens=63, total_tokens=75)}"

'Currently, the weather in Islamabad is 72 degrees Fahrenheit.'

In [21]:
weather_bot("what's the weather like in Islamabad in celsius and karachi in Fahrenheit")

Response 1 tool calls: [ChatCompletionMessageToolCall(id='call_oqByMHt0tIIA8mcGoaa1ld4N', function=Function(arguments='{"location": "islamabad"}', name='get_weather_through_fastapi'), type='function'), ChatCompletionMessageToolCall(id='call_F11zxztx8iFTVyiOtN3kBQi6', function=Function(arguments='{"location": "karachi"}', name='get_weather_through_fastapi'), type='function')]
messages to get Second response  : [{'role': 'user', 'content': "what's the weather like in Islamabad in celsius and karachi in Fahrenheit"}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_oqByMHt0tIIA8mcGoaa1ld4N', function=Function(arguments='{"location": "islamabad"}', name='get_weather_through_fastapi'), type='function'), ChatCompletionMessageToolCall(id='call_F11zxztx8iFTVyiOtN3kBQi6', function=Function(arguments='{"location": "karachi"}', name='get_weather_through_fastapi'), type='function')]), {'tool_call_id': 'call_oqByMHt0tIIA8mc

"Second response  : {'id': 'chatcmpl-9ZELLCfwbo8LyqOx6t0FYbBjxPqk7', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The weather in Islamabad is 72°F (22°C) and in Karachi is 10°C (50°F).', role='assistant', function_call=None, tool_calls=None))], 'created': 1718183643, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_482d920018', 'usage': CompletionUsage(completion_tokens=23, prompt_tokens=126, total_tokens=149)}"

'The weather in Islamabad is 72°F (22°C) and in Karachi is 10°C (50°F).'